In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
from pathlib import Path
import anndata as ad
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "/home/ylu/project/spateo-release_new")
import spateo_dev as st
sys.path.insert(0, "/home/ylu/project")
from utils import *

os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'

%load_ext autoreload
%autoreload 2

2024-07-11 23:59:08.910696: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-11 23:59:09.098101: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64
2024-07-11 23:59:09.098125: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-07-11 23:59:09.868475: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot ope

In [2]:
data_folder = "./results/split_slices_CCF/"
results_folder = "./results/Spateo_global/"
figures_folder = "./results/figures/Spateo_global"
Path(results_folder).mkdir(parents=True, exist_ok=True)

In [3]:
import re
def list_and_sort_files_by_number(directory_path, suffix):
    def extract_number(filename):
        match = re.search(r'\d+', filename)
        return int(match.group()) if match else float('inf')
    try:
        files = [f for f in os.listdir(directory_path) if f.endswith(suffix)]
        sorted_files = sorted(files, key=extract_number)
        return sorted_files
    except FileNotFoundError:
        return f"The directory {directory_path} does not exist."
    except NotADirectoryError:
        return f"The path {directory_path} is not a directory."
    except PermissionError:
        return f"Permission denied to access {directory_path}."

directory_path = data_folder
sorted_files = list_and_sort_files_by_number(directory_path, '.h5ad')

In [4]:
## load the data
from tqdm import tqdm
slices = []
for f in tqdm(sorted_files):
    slices.append(ad.read_h5ad(os.path.join(data_folder, f)))

100%|██████████████████████████| 129/129 [00:22<00:00,  5.65it/s]


In [5]:
spatial_key = "spatial_2D"
anno_key = "cell_type"

In [6]:
## Rotate the data
rotate_key = "spatial_rot"
rotations = np.load("./results/random_rotations.npy", allow_pickle=True)
for i in range(len(slices)):
    slices[i].obsm[rotate_key] = slices[i].obsm[spatial_key][:,:2].copy()
    mean = np.mean(slices[i].obsm[rotate_key], axis=0)
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] - mean
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] @ rotations[i].T + mean
    

In [7]:
method = 'Spateo'
sampling_num = 20000
stats_results_folder = "./results/stats_results/"
transformation = np.load(os.path.join(stats_results_folder, f"{method}_transformation_{sampling_num}.npy"), allow_pickle=True)
align_Rotation = transformation.item()['align_Rotation']
align_translation = transformation.item()['align_translation']

In [8]:
cur_R = np.eye(2)
cur_t = np.zeros(2)
for i in tqdm(range(len(slices))):
    slices[i].obsm['align_spatial_2D'] = slices[i].obsm['spatial_rot'] @ cur_R.T + cur_t
    if i < len(slices)-1:
        R = align_Rotation[i]
        t = align_translation[i]
        cur_t = t @ cur_R.T + cur_t
        cur_R = cur_R @ R

100%|████████████████████████| 129/129 [00:00<00:00, 1172.00it/s]


In [9]:
slices_ref = []
sampling_num = 20000
for s in slices:
    sampline_idx = np.random.choice(s.shape[0], sampling_num, replace=False) if s.shape[0] > sampling_num else np.arange(s.shape[0])
    slices_ref.append(s[sampline_idx,:].copy())

In [ ]:
key_added = "global_align_spatial"
global_align_slices = st.align.morpho_global_optimization(
    models=slices_ref,
    spatial_key='align_spatial_2D',
    key_added="global_align_spatial",
    device="0",
    mode="SN-S",
    max_iter_global=5,
    max_iter=200,
    verbose=False,
    partial_robust_level=25,
    neighbor_size=2,
)

In [23]:
# save the transformation
align_Rotation = []
align_translation = []
for s in global_align_slices:
    R, t=solve_RT_by_correspondence(s.obsm['global_align_spatial'], s.obsm['spatial_2D'])
    align_Rotation.append(R)
    align_translation.append(t)
np.save('./results/stats_results/Spateo_global_transformation.npy', {'align_Rotation': align_Rotation, 'align_translation': align_translation}, allow_pickle=True)